In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xg 

In [175]:
df=pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
dt=pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
sf=pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
df.shape,dt.shape,sf.shape

((188533, 13), (125690, 12), (125690, 2))

In [176]:
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900


In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [12]:
for col in df.columns.drop("id"):
    print(col,df[col].nunique())

brand 57
model 1897
model_year 34
milage 6651
fuel_type 7
engine 1117
transmission 52
ext_col 319
int_col 156
accident 2
clean_title 1
price 1569


In [177]:
df_num=[]
df_cat=[]
for col in df.columns.drop("id"):
  if df[col].dtype=="object":
    df_cat.append(col)
  else:
    df_num.append(col)

In [14]:
for col in df_num:
  print(col,df[col].corr(df["price"]))
  #correlation negative entre milage et prix ce ci indique que le prix baisse suite a l elevation du milage
  #faible correlation entre le model de l annee et le prix

model_year 0.23179481548140743
milage -0.28306749443632345
price 1.0


In [178]:
df.loc[df["accident"].isna(),"accident"]="At least 1 accident or damage reported"

In [179]:
df.loc[(df['accident']=='None reported') & (df['clean_title'].isna()),"clean_title"]="Yes"

In [180]:
df.loc[(df['accident']=='At least 1 accident or damage reported') & (df['clean_title'].isna()),"clean_title"]="No"

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      188533 non-null  object
 11  clean_title   188533 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187737 entries, 0 to 187736
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            187737 non-null  int64 
 1   brand         187737 non-null  object
 2   model         187737 non-null  object
 3   model_year    187737 non-null  int64 
 4   milage        187737 non-null  int64 
 5   fuel_type     182654 non-null  object
 6   engine        187737 non-null  object
 7   transmission  187737 non-null  object
 8   ext_col       187737 non-null  object
 9   int_col       187737 non-null  object
 10  accident      187737 non-null  object
 11  clean_title   187737 non-null  object
 12  price         187737 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.6+ MB


In [181]:
# Flex pattern
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("Flex", case=False, na=False), 'fuel_type'] = 'E85 Flex Fuel'

# Hybrid patterns
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("Dual", case=False, na=False), 'fuel_type'] = 'Hybrid'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("Electric", case=False, na=False), 'fuel_type'] = 'Hybrid'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("Battery", case=False, na=False), 'fuel_type'] = 'Hybrid'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("kW", case=False, na=False), 'fuel_type'] = 'Hybrid'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("AH", case=False, na=False), 'fuel_type'] = 'Hybrid'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['brand'].str.contains("Tesla", case=False, na=False), 'fuel_type'] = 'Hybrid'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['model'].str.contains("Hybrid", case=False, na=False), 'fuel_type'] = 'Hybrid'

# Gasoline patterns
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("GDI", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("PDI", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("OHV", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("MPFI", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("TFSI", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("DOHC", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("Gasoline", case=False, na=False), 'fuel_type'] = 'Gasoline'
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & (df['model_year'] < 2020), 'fuel_type'] = 'Gasoline'

# Diesel patterns
df.loc[(df['fuel_type'].isna()) | df['fuel_type'].isin(['–', 'not supported']) & df['engine'].str.contains("Diesel", case=False, na=False), 'fuel_type'] = 'Diesel'


In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     188533 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      188533 non-null  object
 11  clean_title   188533 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187737 entries, 0 to 187736
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            187737 non-null  int64 
 1   brand         187737 non-null  object
 2   model         187737 non-null  object
 3   model_year    187737 non-null  int64 
 4   milage        187737 non-null  int64 
 5   fuel_type     187737 non-null  object
 6   engine        187737 non-null  object
 7   transmission  187737 non-null  object
 8   ext_col       187737 non-null  object
 9   int_col       187737 non-null  object
 10  accident      187737 non-null  object
 11  clean_title   187737 non-null  object
 12  price         187737 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.6+ MB


In [183]:
le=LabelEncoder()

In [186]:
for col in df_cat:
    df[col]=le.fit_transform(df[col])

In [187]:
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,31,495,2007,213000,2,116,38,312,71,1,1,4200
1,1,28,930,2002,143250,2,366,38,263,10,0,1,4999
2,2,9,1575,2002,136731,1,640,38,38,71,1,1,13900
3,3,16,758,2017,19500,2,863,49,29,14,1,1,45000
4,4,36,1077,2021,7388,2,259,23,29,10,1,1,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,8,604,2017,49000,2,866,49,304,10,1,1,27500
188529,188529,36,206,2018,28600,2,770,31,304,14,0,1,30000
188530,188530,36,223,2021,13650,2,921,23,304,14,1,1,86900
188531,188531,3,1471,2022,13895,2,512,1,82,14,1,1,84900


In [188]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=["id","price"]),df["price"],test_size=0.3, random_state=42)

In [190]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 10, seed = 123) 
  
# Fitting the model 
xgb_r.fit(X_train, y_train) 
  
# Predict the model 
pred = xgb_r.predict(X_test) 
  
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, pred)) 
print("RMSE : % f" %(rmse))

RMSE :  66554.086258


In [191]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [192]:
dt.loc[dt["accident"].isna(),"accident"]="At least 1 accident or damage reported"

In [193]:
dt.loc[(dt['accident']=='None reported') & (dt['clean_title'].isna()),"clean_title"]="Yes"

In [194]:
dt.loc[(dt['accident']=='At least 1 accident or damage reported') & (dt['clean_title'].isna()),"clean_title"]="No"

In [195]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      125690 non-null  object
 11  clean_title   125690 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [148]:
dt["fuel_type"].unique()

array(['Gasoline', 'Hybrid', 'Diesel', 'E85 Flex Fuel', nan,
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [196]:
# Flex pattern
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("Flex", case=False, na=False), 'fuel_type'] = 'E85 Flex Fuel'

# Hybrid patterns
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("Dual", case=False, na=False), 'fuel_type'] = 'Hybrid'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("Electric", case=False, na=False), 'fuel_type'] = 'Hybrid'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("Battery", case=False, na=False), 'fuel_type'] = 'Hybrid'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("kW", case=False, na=False), 'fuel_type'] = 'Hybrid'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("AH", case=False, na=False), 'fuel_type'] = 'Hybrid'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['brand'].str.contains("Tesla", case=False, na=False), 'fuel_type'] = 'Hybrid'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['model'].str.contains("Hybrid", case=False, na=False), 'fuel_type'] = 'Hybrid'

# Gasoline patterns
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("GDI", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("PDI", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("OHV", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("MPFI", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("TFSI", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("DOHC", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("Gasoline", case=False, na=False), 'fuel_type'] = 'Gasoline'
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & (dt['model_year'] < 2020), 'fuel_type'] = 'Gasoline'

# Diesel patterns
dt.loc[(dt['fuel_type'].isna()) | dt['fuel_type'].isin(['–', 'not supported']) & dt['engine'].str.contains("Diesel", case=False, na=False), 'fuel_type'] = 'Diesel'


In [197]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     125690 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      125690 non-null  object
 11  clean_title   125690 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [198]:
for col in df_cat:
    dt[col]=le.fit_transform(dt[col])

In [199]:
dt

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,26,1388,2015,98000,2,326,16,302,10,1,1
1,188534,26,1375,2020,9142,3,787,31,261,14,1,1
2,188535,14,636,2022,28121,2,541,3,302,57,1,1
3,188536,3,182,2016,61258,2,193,39,259,14,1,1
4,188537,3,181,2018,59000,2,365,38,127,14,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,36,759,2014,83315,2,732,23,29,14,1,1
125686,314219,3,1248,2019,29336,2,511,39,302,14,1,1
125687,314220,3,184,2012,77634,2,673,38,29,14,1,1
125688,314221,3,1244,2012,112000,2,673,38,29,14,1,1


In [202]:
x=df.drop(columns=["id","price"])
y=df["price"]

In [203]:
x

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,31,495,2007,213000,2,116,38,312,71,1,1
1,28,930,2002,143250,2,366,38,263,10,0,1
2,9,1575,2002,136731,1,640,38,38,71,1,1
3,16,758,2017,19500,2,863,49,29,14,1,1
4,36,1077,2021,7388,2,259,23,29,10,1,1
...,...,...,...,...,...,...,...,...,...,...,...
188528,8,604,2017,49000,2,866,49,304,10,1,1
188529,36,206,2018,28600,2,770,31,304,14,0,1
188530,36,223,2021,13650,2,921,23,304,14,1,1
188531,3,1471,2022,13895,2,512,1,82,14,1,1


In [205]:
x["clean_title"]=pd.to_numeric(x["clean_title"])

In [206]:
xgb_r.fit(x,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [207]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   brand         188533 non-null  int64
 1   model         188533 non-null  int64
 2   model_year    188533 non-null  int64
 3   milage        188533 non-null  int64
 4   fuel_type     188533 non-null  int64
 5   engine        188533 non-null  int64
 6   transmission  188533 non-null  int64
 7   ext_col       188533 non-null  int64
 8   int_col       188533 non-null  int64
 9   accident      188533 non-null  int64
 10  clean_title   188533 non-null  int64
dtypes: int64(11)
memory usage: 15.8 MB


In [208]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   id            125690 non-null  int64
 1   brand         125690 non-null  int64
 2   model         125690 non-null  int64
 3   model_year    125690 non-null  int64
 4   milage        125690 non-null  int64
 5   fuel_type     125690 non-null  int64
 6   engine        125690 non-null  int64
 7   transmission  125690 non-null  int64
 8   ext_col       125690 non-null  int64
 9   int_col       125690 non-null  int64
 10  accident      125690 non-null  int64
 11  clean_title   125690 non-null  int64
dtypes: int64(12)
memory usage: 11.5 MB


In [209]:
id=dt["id"]
    

In [210]:
x_t=dt.drop(columns="id")

In [211]:
pred=xgb_r.predict(x_t)

In [212]:
output = pd.DataFrame({'id': id, 'price': pred})
output.to_csv('submissiond.csv', index=False)